### <strong>主題:
啤酒評論評分預測 - 分類模型訓練結果測試
### <strong>說明:
花了不少時間終於完成模型的訓練了，接下來就剩下最後一個重要的步驟 - 驗證模型了，這是一 <br />
個相當重要的部分，如何從訓練結果的驗證中評估模型的好壞，並進一步得知如何改善模型是實務 <br />
中必經的一個過程。

### <strong>題目
1. 建構出預測模型的框架，並且載入上次訓練完畢的模型
2. 以預測模型評估先前作業所分割出資料及的各項評分
3. 預測完成之後計算每出各項評分的準確度(accuracy)與解釋力(R square)

In [1]:
# 連接個人資料 讀取 ＰＴＴ 訓練資料和儲存模型
#先連接自己的GOOGLE DRIVE 為了要儲存資料和訓練模型
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os

# Current directory
print(os.getcwd())

# change directory
os.chdir('/content/drive/MyDrive/python_training/NLP100Days-part2/project_2_4/')
print(os.getcwd())

/content
/content/drive/MyDrive/python_training/NLP100Days-part2/project_2_4


In [3]:
!pip install torch
!pip install transformers
#!pip install -q transformers
#!pip install transformers==3
# 設定 torchtext 版本 安裝完必須重新啟動執行階段
!pip install torchtext==0.6.0
#!pip install -r requirements.txt

In [4]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import torch
from torch import nn
from tqdm import tqdm
from transformers import BertModel, BertTokenizer
import sklearn.metrics as metrics

import warnings
warnings.filterwarnings('ignore')

In [5]:
PRE_TRAINED_MODEL_NAME = "bert-base-cased"
#PRE_TRAINED_MODEL_CONFIG = BertConfig.from_pretrained(PRE_TRAINED_MODEL_NAME)
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
TOKENIZER = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
MAX_LEN = 255

In [6]:
class BeerRateClassifier(nn.Module):
    """
    啤酒評論評分分類模型的主體
    Beer sentiment main model for review sentiment analyzer
    """
    def __init__(self,
                 apperance_n_classes,
                 aroma_n_classes,
                 overall_n_classes,
                 palate_n_classes,
                 taste_n_classes,
                ):
        super(BeerRateClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
        # 須完成之部分
        self.drop = nn.Dropout(p=0.2)
        self.apperance_out = nn.Linear(self.bert.config.hidden_size, apperance_n_classes)
        self.aroma_out = nn.Linear(self.bert.config.hidden_size, aroma_n_classes)
        self.overall_out = nn.Linear(self.bert.config.hidden_size, overall_n_classes)
        self.palate_out = nn.Linear(self.bert.config.hidden_size, palate_n_classes)
        self.taste_out = nn.Linear(self.bert.config.hidden_size, taste_n_classes)
 
    def load_model(self, path):
        """
        載入先前訓練好的權重檔
        """
        # 須完成之部分
        self.load_state_dict(torch.load(path, map_location=DEVICE))

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        output = self.drop(outputs['pooler_output']) ##outputs['pooler_output']
        apperance_output = self.apperance_out(output)
        aroma_output = self.aroma_out(output)
        overall_output = self.overall_out(output)
        palate_output = self.palate_out(output)
        taste_output = self.taste_out(output)
        return {
            "apperance": apperance_output,
            "aroma": aroma_output,
            "overall": overall_output,
            "palate": palate_output,
            "taste": taste_output,
        }
    
    def predicts(self, text):
        """
        主要的分類器，將input電影評論輸入模型，將輸出轉化為預測評分
        make prediction according to the text with the given model
        """
        encoding = TOKENIZER.encode_plus(
            text,
            add_special_tokens=True,
            max_length=MAX_LEN,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        # 須完成之部分
        aspect_outputs = self.forward(
            input_ids=encoding['input_ids'],
            attention_mask=encoding['attention_mask']
        )

        #torch.max(output, dim=1)[1].item()
        _, apperance_preds = torch.max(aspect_outputs["apperance"], dim=1)
        _, aroma_preds = torch.max(aspect_outputs["aroma"], dim=1)
        _, overall_preds = torch.max(aspect_outputs["overall"], dim=1)
        _, palate_preds = torch.max(aspect_outputs["palate"], dim=1)
        _, taste_preds = torch.max(aspect_outputs["taste"], dim=1)

        return {"apperance_preds": int(apperance_preds),
                "aroma_preds": int(aroma_preds),
                "overall_preds": int(overall_preds),
                "palate_preds": int(palate_preds),
                "taste_preds": int(taste_preds)
               }

In [7]:
""" 分類模型 """
# 須完成之部分
MODEL = BeerRateClassifier(4, 4, 4, 4, 4)
MODEL.load_model(os.path.join('data', 'best_model_state.bin'))

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
VAL = pd.read_json("./data/test_set_lng.json")
VAL = VAL.sample(frac=1).reset_index(drop=True)
#TEST = list(VAL['review/text'])[:200] + list(VAL['review/text'])[-200:]
TEST = pd.read_json(os.path.join('data', 'test_set.json'), encoding='utf-8')
# 須完成之部分
data = []
for _, row in tqdm(TEST.iterrows(), total=TEST.shape[0]): #.iterrows()
    if row['review/text'] is not None:
        text = row['review/text'].replace('<br />', ' ')
    else:
        text = ''
    predict = MODEL.predicts(text)
    data.append(predict)

100%|██████████| 5000/5000 [54:38<00:00,  1.53it/s]


In [11]:
TEST.head(3)

,index,beer/ABV,beer/beerId,beer/brewerId,beer/name,beer/style,review/appearance,review/aroma,review/overall,review/palate,review/taste,review/text,review/timeStruct,review/timeUnix,user/ageInSeconds,user/birthdayRaw,user/birthdayUnix,user/gender,user/profileName,review_appearance,review_aroma,review_overall,review_palate,review_taste,text_length
0,6248,5.4,48555,21516,Kirkland Signature Pale Ale,American Pale Ale (APA),NaN,NaN,NaN,NaN,NaN,A- Crystal clear amber body with a course soda...,"{'min': 41, 'hour': 21, 'mday': 30, 'sec': 19,...",1306791679,NaN,None,NaN,None,gpogo,3,3,3,3,3,65
1,8691,5.6,1123,395,Dark Star Porter,English Porter,NaN,NaN,NaN,NaN,NaN,"Thin lacing, color is more like a very dark br...","{'min': 25, 'hour': 14, 'mday': 21, 'sec': 7, ...",1219328707,NaN,None,NaN,None,jordynelson,3,3,3,3,3,55
2,40328,4.9,6769,1315,Weltenburger Kloster Urtyp Hell,Munich Helles Lager,NaN,NaN,NaN,NaN,NaN,"This is a pretty good, but rather sweet, basic...","{'min': 41, 'hour': 6, 'mday': 13, 'sec': 24, ...",1260686484,NaN,None,NaN,None,Offa,3,3,3,3,3,64


In [12]:
data[:3]

[{'apperance_preds': 1,
  'aroma_preds': 1,
  'overall_preds': 1,
  'palate_preds': 1,
  'taste_preds': 1},
 {'apperance_preds': 3,
  'aroma_preds': 3,
  'overall_preds': 3,
  'palate_preds': 3,
  'taste_preds': 3},
 {'apperance_preds': 2,
  'aroma_preds': 3,
  'overall_preds': 3,
  'palate_preds': 3,
  'taste_preds': 3}]

In [13]:
data[0]

{'apperance_preds': 1,
 'aroma_preds': 1,
 'overall_preds': 1,
 'palate_preds': 1,
 'taste_preds': 1}

In [16]:
data[0].keys()

dict_keys(['apperance_preds', 'aroma_preds', 'overall_preds', 'palate_preds', 'taste_preds'])

In [15]:
list(data[0].keys())

['apperance_preds',
 'aroma_preds',
 'overall_preds',
 'palate_preds',
 'taste_preds']

In [27]:
preds_key= ['apperance_preds','aroma_preds','overall_preds','palate_preds','taste_preds']
test_key = ['review_appearance','review_aroma','review_overall','review_palate','review_taste']

#for aspect in aspects:
for i in range(5):
    preds = [d[f"{preds_key[i]}"] for d in data]
    #labels = [row[f"review_{aspect}"] for _, row in TEST.iterrows()]
    labels = [row[f"{test_key[i]}"] for _, row in TEST.iterrows()]

    print(f"---{aspect}---")
    print(f"Accuracy: {metrics.accuracy_score(labels, preds)}")
    print(f"R2 Score: {metrics.r2_score(labels, preds)}")

---appearance---
Accuracy: 0.5698
R2 Score: 0.0
---appearance---
Accuracy: 0.6304
R2 Score: 0.0
---appearance---
Accuracy: 0.6162
R2 Score: 0.0
---appearance---
Accuracy: 0.571
R2 Score: 0.0
---appearance---
Accuracy: 0.6304
R2 Score: 0.0
